In [6]:
"""
Koden henter laksedata fra SSB og gjør dataen om til en tabell for alle verdiene som brukes videre i prosjektet ved hjelp av Pandas.
"""

import requests
import pandas as pd
from itertools import product

# URL til SSB API for tabell 03024
API_URL = "https://data.ssb.no/api/v0/no/table/03024/"

def fetch_data(api_url=API_URL):
    """Henter data fra SSB API og returnerer JSON-responsen."""
    query = {
        "query": [],
        "response": {"format": "json-stat2"}
    }
    response = requests.post(api_url, json=query)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Feil ved henting av data: {response.status_code}\n{response.text}")

def process_data(data):
    """Prosesserer JSON-data og returnerer en formatert Pandas DataFrame."""
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
   

    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    all_combinations = list(product(*dimensions))
    

    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values  
    

    df_pivot = df.pivot_table(index="Tid", columns=["VareGrupper2", "ContentsCode"], values="Value", aggfunc="sum").reset_index()
    
    
    df_pivot.columns = ["År og ukenr.", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)", "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]
    
    return df_pivot

def main():
    """Hovedfunksjon for å hente og prosessere data."""
    try:
        data = fetch_data()
        df_pivot = process_data(data)
        print(df_pivot.to_string(index=False))
        print("Data hentet, sortert og formatert for visning.")
    except Exception as e:
        print(e)

if __name__ == "__main__":
    main()

HTTPSConnectionPool(host='data.ssb.no', port=443): Max retries exceeded with url: /api/v0/no/table/03024/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x10d71f610>: Failed to resolve 'data.ssb.no' ([Errno 8] nodename nor servname provided, or not known)"))


In [8]:
from IPython.display import display, HTML
import numpy as np

# Sørg for at du har en 'År'-kolonne i df_pivot
if "År" not in df_pivot.columns:
    df_pivot["År"] = df_pivot["Tid"].str[:4].astype(int)

# Velg numeriske kolonner
numeriske_kolonner = [
    "Fersk laks - Kilospris", 
    "Fersk laks - Vekt (tonn)", 
    "Frosset laks - Kilospris", 
    "Frosset laks - Vekt (tonn)"
]

# Liste for å lagre resultater
statistikk_per_år = []

# Gå gjennom hvert år
for år in sorted(df_pivot["År"].unique()):
    df_år = df_pivot[df_pivot["År"] == år]
    for kol in numeriske_kolonner:
        verdier = df_år[kol].dropna()
        statistikk_per_år.append({
            "År": år,
            "Variabel": kol,
            "Gjennomsnitt": round(np.mean(verdier), 2),
            "Median": round(np.median(verdier), 2),
            "Standardavvik": round(np.std(verdier), 2)
        })

#Lag en DataFrame av resultatene
df_stat_år = pd.DataFrame(statistikk_per_år)



display(HTML(f"""
<div style="height:300px; overflow:auto; border:1px solid lightgray;">
{df_stat_år.to_html()}
</div>
"""))



# Bruk pivot_table for å gjøre det oversiktlig
pivot = df_stat_år.pivot_table(
    index="År", 
    columns="Variabel", 
    values=["Gjennomsnitt", "Median", "Standardavvik"]
)

# Sorter kolonne-nivåene for penere utskrift
pivot = pivot.sort_index(axis=1, level=0)

# Vis med scroll hvis ønskelig
from IPython.display import display, HTML

display(HTML(f"""
<div style="height:350px; overflow:auto; border:1px solid lightgray; padding:5px;">
{pivot.round(2).to_html()}
</div>
"""))

for mål in ["Gjennomsnitt", "Median", "Standardavvik"]:
    print(f"\n🔹 {mål}")
    visning = df_stat_år.pivot(index="År", columns="Variabel", values=mål).round(2)
    display(HTML(visning.to_html()))



NameError: name 'df_pivot' is not defined

In [ ]:
"""
En samlet oversikt over statistisk analyse av laksedataene fra SSB, som viser gjennomsnitt, median og standardavvik for hver variabel over alle år.
"""

import numpy as np
import pandas as pd

#De ulike kolonnene i datasettet som vi bruker
numeriske_kolonner = [
    "Fersk laks - Kilospris", 
    "Fersk laks - Vekt (tonn)", 
    "Frosset laks - Kilospris", 
    "Frosset laks - Vekt (tonn)"
]

#Regner ut statistikken for alle årene til sammen
samlet_statistikk = []

for kol in numeriske_kolonner:
    verdier = df_pivot[kol].dropna()
    samlet_statistikk.append({
        "Variabel": kol,
        "Gjennomsnitt": round(np.mean(verdier), 2),
        "Median": round(np.median(verdier), 2),
        "Standardavvik": round(np.std(verdier), 2)
    })


df_samlet_statistikk = pd.DataFrame(samlet_statistikk)
df_samlet_statistikk.set_index("Variabel", inplace=True)

#visualieres med en tabell som kan scrolle, bruker derfor HTML
from IPython.display import display, HTML

display(HTML(f"""
<div style="width:60%; padding:8px; border:1px solid #ddd; overflow:auto;">
{df_samlet_statistikk.to_html()}
</div>
"""))


,Gjennomsnitt,Median,Standardavvik
Variabel,,,
Fersk laks - Kilospris,45.55,37.94,23.12
Fersk laks - Vekt (tonn),13065.61,13615.50,5762.24
Frosset laks - Kilospris,45.98,40.68,20.77
Frosset laks - Vekt (tonn),625.02,564.00,309.08
